# Imports & random seed


In [ ]:
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_algorithms
!pip install pylatexenc
!pip install qiskit_ibm_runtime
!pip install qiskit_nature qiskit qiskit.algorithms qiskit_aer
!pip install pyscf


In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Parameter
from qiskit_aer.primitives import Sampler, Estimator
from qiskit_algorithms.optimizers import SLSQP, SPSA
import pylatexenc
from qiskit_ibm_runtime.fake_provider import FakeKolkata
from qiskit.circuit.library import EfficientSU2
import numpy as np
import qiskit_nature
from qiskit_algorithms import NumPyMinimumEigensolver, VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
from qiskit_aer.primitives import Estimator
from qiskit_aer.noise import NoiseModel
from qiskit_algorithms.optimizers import SLSQP, SPSA, COBYLA
from qiskit_aer.noise import NoiseModel


np.random.seed(999999)

# Used functions

In [ ]:
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name="c")
    qc = QuantumCircuit(qr, cr)
    qc.u(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

In [ ]:
def get_qubit_op(molecule):
    '''
    Uses ParityMapper
    '''
    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=num_particles)
    # mapper = JordanWignerMapper()
    qubit_op = mapper.map(problem.second_q_ops()[0])
    return qubit_op, num_particles, num_spatial_orbitals, problem, mapper

In [ ]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

# Class MoleculeManager

In [ ]:
class MoleculeManager():
  '''
  MoleculeManager\n

  '''
  def __init__(
      self,
      molecule: MoleculeInfo,
      optimizer=COBYLA(maxiter=500, tol=0.0001),

  ):
    self.molecule = molecule
    self.optimizer = optimizer


  def SetAtomDist(self, dist: float):
    self.molecule.coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0])


  def FindEnergyIdeal(self, print_ansatz=True, print_circuit_info=True):
    exact_energies = []
    vqe_energies = []
    optimizer = SLSQP(maxiter=10)
    noiseless_estimator = Estimator(approximation=True)
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(self.molecule)

    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies[0].real)
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    ansatz = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    vqe = VQE(
        noiseless_estimator,
        ansatz,
        optimizer,
        initial_point=[0] * ansatz.num_parameters,
    )
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    if (print_ansatz):
      display(ansatz.decompose().decompose().draw(fold=-1))
    if (print_circuit_info):
      print(f'ansatz.depth = {ansatz.depth()}')
      print(f'num of qubits = {ansatz.num_qubits}')
    print(result)
    print(
        f"### TODO! what's the difference between VQE Result and Exact energy?\n",
        f"Interatomic Distance: {np.linalg.norm([self.molecule.coords[0],self.molecule.coords[1]])}\n",
        f"VQE Result: {vqe_result:.5f}\n",
        f"Exact Energy: {exact_energies[-1]:.5f}\n",
    )


  def FindEnergyNoisy(self):
    exact_energies = []
    vqe_energies = []
    device = FakeKolkata()
    coupling_map = device.configuration().coupling_map
    noise_model = NoiseModel.from_backend(device)
    noisy_estimator = Estimator(
        backend_options={"coupling_map": coupling_map, "noise_model": noise_model}
    )
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(self.molecule)
    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies)

    print("Exact Result:", result.total_energies)
    optimizer = SPSA(maxiter=100)
    var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")
    vqe = VQE(noisy_estimator, var_form, optimizer)
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies
    print("VQE Result:", vqe_result)


In [ ]:
dist = 1
LiH = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

MM = MoleculeManager(
    molecule=LiH
)

In [ ]:
MM.FindEnergyIdeal(print_ansatz=False)

ansatz.depth = 1
num of qubits = 8
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -9.371552953208
  - computed part:      -1.174236549311
  - FreezeCoreTransformer extracted energy part: -8.197316403898
~ Nuclear repulsion energy (Hartree): 1.58753163276
> Total ground state energy (Hartree): -7.784021320448
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [1.88972612  0.0  0.0]
 
### TODO! what's the difference between VQE Result and Exact energy?
 Interatomic Distance: 1.0
 VQE Result: -7.78402
 Exact Energy: -7.78402



In [ ]:
MM.FindEnergyNoisy()

<ipython-input-19-3c32b9b748b8>:59: DeprecationWarning: V1 fake backends are deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use V2 fake backends instead.
  device = FakeKolkata()
<ipython-input-21-6e7b647485ab>:1: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  MM.FindEnergyNoisy()


Exact Result: [-7.78402132]
VQE Result: [-6.58026361]
